In [1]:
#| echo: false
#!pip install /data/home/prokopis/ilsp-nlp/ilsp_nlp/
%load_ext autoreload
%autoreload 2

# Create datasets and upload them to argilla

In [2]:
#| output: false
import joblib
from pathlib import Path

from IPython.display import display_markdown
from IPython.display import display, HTML
from typing import List, Dict
from tqdm import tqdm
import pandas as pd
import regex as re
import random
import logging
import sys
import pprint
import difflib
import urllib
from tqdm import tqdm
import requests
import json
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import io, base64
from datetime import datetime

import numpy as np
import argilla as rg
from collections import Counter

from datasets import Dataset, DatasetDict, DatasetInfo, Features, Sequence, Value
from datasets import load_from_disk
from huggingface_hub import DatasetCard, DatasetCardData

sns.set_theme()
SPACE = " "
NL="\n"

import logging
from logging.handlers import RotatingFileHandler
import sys

log_file = Path.home() / "logs" / ".log"
def setup_logger(log_file="out.log", max_bytes=10*1024*1024, backup_count=5):
    """
    Sets up a logger that logs INFO and above to stdout and everything to a
    rolling file.

    Args:
        log_file: The name of the log file (default: "out.log").
        max_bytes: The maximum size of the log file before it rolls over
            (default: 10MB).
        backup_count: The number of backup log files to keep (default: 5).
    """
    # Create a logger
    logger = logging.getLogger(__name__)  # Or use a specific name if you prefer
    logger.setLevel(logging.DEBUG)  # Set the lowest level to be handled

    # Create a formatter
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s [%(name)s:%(lineno)d]')

    # Create a rolling file handler (all levels)
    file_handler = RotatingFileHandler(log_file,
                                       maxBytes=max_bytes,
                                       backupCount=backup_count)
    file_handler.setLevel(logging.DEBUG)  # Log all levels to the file
    file_handler.setFormatter(formatter)

    # Create a handler for stdout (INFO and above)
    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setLevel(logging.INFO)
    stdout_handler.setFormatter(formatter)

    # Add the handlers to the logger
    logger.handlers.clear()
    logger.addHandler(stdout_handler)
    # logger.addHandler(file_handler)

    return logger

logger = setup_logger()
logger.info("Reached")

2025-05-11 14:16:34,297 - INFO - Reached [__main__:81]


## Users and workspaces

In [ ]:
client = rg.Argilla(
    api_url="https://nlp.ilsp.gr/argilla/",
    api_key="9TYK93yXHE0NPi7x59JLvkv4--sptW-jm4iH5sosOX7st6ZkERNIeE_ATl6AFRrQXmzqvNy88dUQJRG4LUoKNcspayyBRMcUgzm-ie3G_24" 
    #api_key="argilla.apikey"        
)


for user in client.users:
    user_to_delete = "xxxxxxx"
    if user.username==user_to_delete:        
        logger.debug(f"User {user}")
        print(f"username: {user.username}, role: {user.role}, password: {user.password}")
        user_to_delete = client.users(user_to_delete)
        deleted_user = user_to_delete.delete()        

In [4]:
recreate = False
users = client.users
if recreate:
    users_to_create = [
    ]    
    for user_to_create in users_to_create:
        user = client.users(user_to_create["username"])
        if user is not None:
            continue
        rg.User(
            username=user_to_create["username"],
            password=user_to_create["password"],
            first_name=user_to_create["first_name"],
            last_name=user_to_create["last_name"],
            role=user_to_create["role"],            
        ).create()



In [8]:
import traceback
recreate = True
workspaces = client.workspaces
if recreate:    
    workspaces_to_create = [
            #{"name":"pkyriazi", "users_to_add":["pkyriazi", "prokopis"]},
            # {"name":"prokopis", "users_to_add":["prokopis"]}
        {"name":"asep", "users_to_add":["prokopis", "pkyriazi"]}
                   ]
    for workspace_to_create in workspaces_to_create:
        try:
            created_workspace = rg.Workspace(name=workspace_to_create["name"]).create()
            #logger.debug(created_workspace)
        except:
            pass
            #traceback.print_exc()
            
        for user_to_add in workspace_to_create["users_to_add"]:
            user_to_add = client.users(user_to_add)
            try: 
                #logger.debug(user_to_add)
                user_to_add.add_to_workspace(client.workspaces(workspace_to_create["name"]))
                #added_user = rg.Workspace(name=workspace_to_create["name"]).add_user(user_to_add)
            except:
                pass

for workspace in workspaces:    
    for user in workspace.users:
        pass
        #logger.info(f"Workspace: {workspace.name}, User:{user.username}, UserId:{user.id}")

## Recreate

In [9]:
recreate = True
dataset_name = "asep_emails"
workspace_name = "asep"
input_json = "/home/pkyriazi/projects/presidio/ilsp-athenarc-asep-anonymizer/notebooks/ASEP_anonymized_pseudo_dataset.json"
input_json = "/home/pkyriazi/projects/presidio/ilsp-athenarc-asep-anonymizer/notebooks/ASEP_anonymized_dataset.json"

if recreate:
    settings = rg.Settings(
        guidelines="ΑΣΕΠ artificial emails",
        fields=[
            rg.TextField(
                name="subject",
                title="Subject",
            ),
            rg.TextField(
                name="full_text",
                title="Full text",
                required=True,                
            ),
            # rg.TextField(
            #     name="masked",
            #     title="Masked",
            #     required=True,
            # ),
        ],
        metadata=[
             rg.TermsMetadataProperty(
                name="message_type",
                options=["message", "response"],
                title="Message type",
                visible_for_annotators=True,
            ),
           rg.TermsMetadataProperty(
                name="contest_value",                
                title="Contest value",
                visible_for_annotators=True,
            ),
            rg.TermsMetadataProperty(
                name="department_name",
                title="Department name",
                visible_for_annotators=True,
            ),
            rg.TermsMetadataProperty(
                name="help_categories",
                title="Help categories",
                visible_for_annotators=True,
            ),
            rg.TermsMetadataProperty(
                name="parent_categories",
                title="Parent categories",
                visible_for_annotators=True,
            ),
            rg.IntegerMetadataProperty(
                name="ticket_id",
                title="Ticket id",
                visible_for_annotators=True,
            ),
            rg.IntegerMetadataProperty(
                name="number_id",
                title="Number id",
                visible_for_annotators=True,
            ),
            rg.TermsMetadataProperty(
                name="question_date",
                title="Question date",
                visible_for_annotators=True,
            ),
        ],
        questions=[
            rg.SpanQuestion(
                name="pii",
                title="PII",
                field="full_text",
                labels=["PERSON", 
                        "ADDRESS",
                        "EMAIL_ADDRESS",
                        "PHONE_NUMBER",                        
                        "AFM",
                        "AMKA",
                        "ADT",                        
                        "LOCATION",                        
                        "GPE", 
                        "ORG", 
                        "MISC", 
                       ],
                description = "Annotate text spans that should be anonymized in the full text field with one of these labels.",
                allow_overlapping=False,
            ),
        ],
    )
    
    dataset = rg.Dataset(
        name=dataset_name,
        workspace=workspace_name, 
        settings=settings,
        client=client,
    )
    
    for cdataset in client.datasets:
        if cdataset.name==dataset_name:
            cdataset.delete()
    dataset.create()
    
    # Open and read the JSON file
    with open(input_json, 'r') as file:
        data = json.load(file)    
    
    records = list()
    questions = [q.name for q in dataset.questions ]
    user_id =  client.users("pkyriazi").id
    for pidx, p in enumerate(data):
        if (pidx ) % 1000 == 0:
            logger.info(f"Processing record #{pidx+1}")

        record = rg.Record(
            fields = {
                    "full_text": p["full_text"],
                    "subject": "missing_subject",
                },
            metadata = {},
        )
        for span in p["spans"]:
            record.suggestions.add(rg.Suggestion("pii", value=[{"start": span["start_position"], "end": span["end_position"], "label": span["entity_type"]}]))            

        records.append(record)

    dataset.records.log(records)



2025-05-11 14:25:23,093 - INFO - Processing record #1 [__main__:113]


INFO:__main__:Processing record #1


DatasetRecords: The provided batch size 256 was normalized. Using value 250.

Sending records...: 100%|██████████| 1/1 [00:01<00:00,  1.72s/batch]


## Backup

## Restore

## Update records